# Homework 1

## 1.1

Introspection might give a little insight into the congnitive processes, but I don't think it is valuable for modelling the mind. If you really wanted to model the human mind, monitoring it through experiments or through external observation would seem to be more powerful. The problem with instrospection is that it is subjective, despite the attempts for the practitioner to be unbiased in their analysis. It seems that humans tend to overestimate their thought processes/intelligence, when it really isn't all that complicated. Rather, it's the simple, unconscious subprocesses that are impressive aspects of the brain.

## 1.2

I formulated the problem by representing the path of the travelling salesman as an ordered array of locations that loops around to the start. The locations are represented using their (x, y) coordinates, and the distances between two locations is calculated using the straight distance between the two points. The value function calculates the total distance of the path using these metrics (and makes it negative for max value processing). The actions are unique combinations of two locations that get swapped in the result function. The goal test always returns false because there is no known solution.

In [25]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule
import itertools
import math

class TSP(Problem):
    def __init__(self, locations, initial):
        self.locations = locations
        self.initial = initial
     
    def actions(self, state): 
        return list([c for c in itertools.combinations(state, 2)])
         
    def goal_test(self, state): 
        return False
    
    def result(self, state, move):
        new_state = state[:]
        i, j = new_state.index(move[0]), new_state.index(move[1])
        new_state[i], new_state[j] = new_state[j], new_state[i]
        return new_state
    
    def value(self, state):
        value = 0
        for i in range(len(state)):
            a, b = self.locations[state[i]], self.locations[state[(i + 1) % len(state)]]
            value += math.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)
        return -value

I simply used the lab code from the Romania problem as my city formulation. I simply use the (x, y) coordinates of locations, but the problem could easily be switched to use a dictionary of distances between each city. The initial state is generated by taking the keys of the dictionary and randomly shuffling them so we start with a random path.

In [35]:
import random
locations = dict(
    arad=(91, 492), 
    bucharest=(400, 327), 
    craiova=(253, 288), 
    dobreta=(165, 299),
    eforie=(562, 293), 
    fagaras=(305, 449), 
    giurgiu=(375, 270), 
    hirsova=(534, 350),
    iasi=(473, 506), 
    lugoj=(165, 379), 
    mehadia=(168, 339), 
    neamt=(406, 537),
    oradea=(131, 571), 
    pitesti=(320, 368), 
    rimnicuvilcea=(233, 410), 
    sibiu=(207, 457),
    timisoara=(94, 410), 
    urziceni=(456, 350), 
    vaslui=(509, 444), 
    zerind=(108, 531))

initial = list(locations.keys())
random.shuffle(initial)
p = TSP(locations, initial)

hill_solution = hill_climbing(p)
print('Hill-climbing:')
print('\tSolution: ' + str(hill_solution))
print('\tValue:    ' + str(-p.value(hill_solution)))
print('\tGoal?     ' + str(p.goal_test(hill_solution)))


annealing_solution = simulated_annealing(
    p,
    exp_schedule(k=20, lam=0.005, limit=15000)
)
print('Simulated annealing:')
print('\tSolution: ' + str(annealing_solution))
print('\tValue:    ' + str(-p.value(annealing_solution)))
print('\tGoal?     ' + str(p.goal_test(annealing_solution)))

Hill-climbing:
	Solution: ['pitesti', 'rimnicuvilcea', 'fagaras', 'neamt', 'iasi', 'vaslui', 'hirsova', 'eforie', 'sibiu', 'oradea', 'zerind', 'arad', 'timisoara', 'lugoj', 'mehadia', 'dobreta', 'craiova', 'giurgiu', 'bucharest', 'urziceni']
	Value:    1947.2136201329545
	Goal?     False
Simulated annealing:
	Solution: ['urziceni', 'eforie', 'hirsova', 'vaslui', 'iasi', 'neamt', 'fagaras', 'pitesti', 'craiova', 'dobreta', 'mehadia', 'lugoj', 'timisoara', 'arad', 'zerind', 'oradea', 'sibiu', 'rimnicuvilcea', 'giurgiu', 'bucharest']
	Value:    1677.3667512731465
	Goal?     False


Simulated annealing consistently performs much better than the hill-climbing. But if random restarts were implemented, the gap in perfomance would shrink a little. Hill-climbing is so dependent on the initial state that it usually ends up at a poor local maximum. But simulated annealing has the potential to explore more areas than just it's starting state. 

## 1.3

Follow the same pattern for this problem.

In [3]:
print('Here\'s yet more code')

Here's yet more code
